In [153]:
import requests, json, os
import psycopg2
from datetime import datetime
from dateutil import tz

In [136]:
requests.__version__

'2.22.0'

In [117]:
DIVVY_URL = 'https://gbfs.divvybikes.com/gbfs/en/station_status.json'

# Make Connection to postgres

## Initialize connection

In [182]:
gcp_sql_username = os.environ.get('gcp_sql_username')
gcp_sql_password = os.environ.get('gcp_sql_password')

conn = psycopg2.connect(user=gcp_sql_username, password=gcp_sql_password,
                        host='localhost', port='5432')

## Query data 

In [183]:
DISPLAY_ROWS = 5

cur = conn.cursor()
cur.execute('SELECT * FROM divvylivedata;')
print("Total rows: {}\nDisplayed rows: {}\n".format(cur.rowcount, DISPLAY_ROWS))

row_counter = 1
row = cur.fetchone()
while row is not None and row_counter <= DISPLAY_ROWS:
    print(','.join([str(v) for v in row]))
    row = cur.fetchone()
    row_counter += 1
 
cur.close()

Total rows: 0
Displayed rows: 5



### Convert unix timestamps into timestamps and consider timezone

In [165]:
utc_timestamp = datetime.utcfromtimestamp(1565246835).strftime('%Y-%m-%d %H:%M:%S')
print(utc_timestamp)

2019-08-08 06:47:15


In [167]:
# METHOD 1: Hardcode zones:
from_zone = tz.gettz('UTC')
to_zone = tz.gettz('America/Chicago')

# # METHOD 2: Auto-detect zones:
# from_zone = tz.tzutc()
# to_zone = tz.tzlocal()

# utc = datetime.utcnow()

utc = datetime.strptime(utc_timestamp, '%Y-%m-%d %H:%M:%S')

# Tell the datetime object that it's in UTC time zone since 
# datetime objects are 'naive' by default
utc = utc.replace(tzinfo=from_zone)

# Convert time zone
central = utc.astimezone(to_zone)

print("Local time in Chicago: ", central)

Local time in Chicago:  2019-08-08 01:47:15-05:00


## Writing data

In [186]:
def get_live_divvy_data():
    # query data from divvy feeds
    res = requests.get(DIVVY_URL)
    
    # serialize data
    jsonres = res.json()
    
    # prepare data
    lst_updt = datetime.utcfromtimestamp(int(jsonres['last_updated'])).strftime('%Y-%m-%d %H:%M:%S')
    
    output_lst = []
    for station_dict in jsonres['data']['stations']:
        try:
            lst_rprt = datetime.utcfromtimestamp(int(station_dict['last_reported'])).strftime('%Y-%m-%d %H:%M:%S')
            stid = int(station_dict['station_id'])
            
            nba = int(station_dict['num_bikes_available'])
            nda = int(station_dict['num_docks_available'])
            nea = int(station_dict['num_ebikes_available'])
            nbd = int(station_dict['num_bikes_disabled'])
            ndd = int(station_dict['num_docks_disabled'])
            
            ifinstl = bool(int(station_dict['is_installed']))
            ifrent = bool(int(station_dict['is_renting']))
            ifrtrn = bool(int(station_dict['is_returning']))
            
            output_lst.append((lst_updt, lst_rprt, stid, 
                               nba, nda, nea, nbd, ndd, 
                               ifinstl, ifrent, ifrtrn))
        except (ValueError, KeyError):
            continue
    
    return output_lst

In [187]:
%%time
sql = """
INSERT INTO divvylivedata (
    last_updated,
    last_reported,
    stationid,
    num_bikes_available,
    num_docks_available,
    num_ebikes_available,
    num_bikes_disabled,
    num_docks_disabled,
    is_installed,
    is_renting, 
    is_returning
)
VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

cur = conn.cursor()
# execute the INSERT statement
cur.executemany(sql, get_live_divvy_data())
# commit the changes to the database
conn.commit()
# close communication with the database
cur.close()

CPU times: user 125 ms, sys: 60 ms, total: 185 ms
Wall time: 36.5 s


## Close connection

In [50]:
if conn:
    conn.close()

# PostgreSQL table creation

```SQL
CREATE TABLE divvylivedata (
    id SERIAL PRIMARY KEY,
    last_updated TIMESTAMP,
    last_reported TIMESTAMP,
    stationid INTEGER,
    num_bikes_available INTEGER,
    num_docks_available INTEGER,
    num_ebikes_available INTEGER,
    num_bikes_disabled INTEGER,
    num_docks_disabled INTEGER,
    is_installed BOOLEAN,
    is_renting BOOLEAN, 
    is_returning BOOLEAN,
    created_at TIMESTAMP NOT NULL DEFAULT NOW()
);
```

### SQL useful command examples

Create postgres table
```SQL
CREATE TABLE divvytemplate (
    id SERIAL PRIMARY KEY,
    feedid INTEGER,
    stationid INTEGER,
    num_bikes_available INTEGER,
    num_docks_available INTEGER
);
```

Add column
```SQL
ALTER TABLE divvytemplate
ADD COLUMN time DATE;
```

Delete column
```SQL
ALTER TABLE divvytemplate DROP COLUMN time;
```

Insert new data into 
```SQL
INSERT INTO divvytemplate (feedid, stationid, num_bikes_available, num_docks_available) 
VALUES
    (1565243017, 2, 20, 19), 
    (1565243017, 3, 4, 48);
```